# Deferred Revenue Forecast

This jupyter notebook contains the details Adobe's deferred revenue forecast model in a more readable, easy to understand format.

The program is run from python directly (versus using a notebook like we have here) and the details are available on my github page www.github.com/davidjsmith44/Deferred_Revenue_Forecast
Note: You will need me to give you access to this repository as it is not public

<b>This is bold text </b>

<i>Italic Text </i>

<blockquote> This is where I have my blockquote text. It will all be indented to make it easier to read </blockquote>

This is not blockquote, but below is a horizontal line
___

<b> Step 1: Process the Base Billings Data </B>


Steps to the program
1. Load up all input data
    - billings history
        - Type A
    - FX rates
    - FX_currency map
    - FX forwards
    - bookings data
 
 
 2. Process the billings data into a dataframe that includes the BU, currency, period and every type of billings based on it's rebill frequency
 
 3. Process the bookings information

4. Forecast the future billings

5. Basic reporting documents

6. Checking for sanity


NOTE: If you plan on using a Jupyter Notebook to run the Deferred Revenue Forecast, you must first download all of the input data to your local directory. I do not yet know how to use a Jupyter Notebook and access data off of the Treasury server.

The input data sits on the Treasury server under Treasury\Financial_Database\Deferred_Revenue\Inputs\Data_YYYY_pMM

There will be 6 files located at this directory that need to be copied to your local drive (preferably where you have installed Anaconda) under a directory called data

## Code Begins here
Below are standard import statements to include all the functionality of numpy, pandas, matplotlib, pickle and a linear regression moodel from sklearn.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import pickle
from math import ceil
from sklearn.linear_model import LinearRegression


## Step 1: Processing Base Billings Data

The billings data comes from tableau and is saved in a file "all_billings_inputs.xlsx" with two sheets of information

"base_billings"
Contains the basic information about all of the billings

In [2]:
df = pd.read_excel('../data/all_billings_inputs.xlsx', sheet_name='base_billings')

In [3]:
df.sample(10)

,Document Currency,Enterprise BU Desc,Frequency,Invoice Fiscal Year Period Desc,Product Config Type,Rev Rec Category,Rule For Bill Date,Sales Type,Subscription Term,Completed Sales ( DC ),Completed Sales
36629,SEK,Print & Publishing,NaN,2015-05,1V,NaN,NaN,RECOGNIZED,0,3420.00,400.48
24279,GBP,Experience Cloud,ONGO,2018-07,NaN,NaN,NaN,DEFERRED,0,0.00,0.00
31136,JPY,Print & Publishing,NaN,2017-08,1Y,D,Y3,DEFERRED,1,822216.00,7297.97
30453,JPY,Experience Cloud,ONGO,2018-10,NaN,NaN,NaN,RECOGNIZED,0,9306650.00,82895.26
44828,USD,Experience Cloud,ACTL,2016-07,NaN,D,YC,DEFERRED,0,70000.00,70000.00
41840,USD,Document Cloud,NaN,2020-04,1Y,D,NaN,DEFERRED,12,4027054.29,4027054.29
17127,EUR,Experience Cloud,ONGO,2015-08,ONORE,D,YH,DEFERRED,0,127112.65,140958.44
26830,JPY,Creative,NaN,2016-01,IDRT,NaN,NaN,RECOGNIZED,0,74080.00,606.71
31097,JPY,Print & Publishing,NaN,2017-04,1V,NaN,NaN,RECOGNIZED,0,215282.00,1916.96
16163,EUR,Experience Cloud,1TME,2017-09,NaN,D,YQ,DEFERRED,0,362.48,428.51


### Changing the column names early since they are inconsistent across other reports

In [4]:
df.rename(index = str, columns = {'Document Currency': 'curr',
                                 'Enterprise BU Desc': 'BU',
                                 'Invoice Fiscal Year Period Desc': 'period',
                                 'Product Config Type': 'config',
                                 'Rev Rec Category': 'rev_req_type',
                                 'Rule For Bill Date': 'rebill_rule',
                                 'Completed Sales ( DC )': 'DC_amount',
                                 'Completed Sales': 'US_amount'}, inplace=True)

### Filter that removes any currency that has  < 10 transactions. 


In [6]:
# creates a list of the currencies and the number of transactions for each currency
vc = df['curr'].value_counts()
print(vc)

USD    12427
EUR     7725
GBP     5714
AUD     5068
JPY     4919
CHF     2389
SEK     2260
DKK     2172
NOK     1854
CAD     1520
HKD      486
BRL      466
RUB      461
KRW      247
CLP      211
ARS      210
COP      210
SGD      209
PEN      190
INR      181
PHP      151
TWD      148
THB      144
MYR      138
IDR      132
NZD       67
ILS       38
TRY       27
SAR        4
BMD        2
AED        1
MXP        1
Name: curr, dtype: int64


In [7]:
# Create variable that is true if the number of transaction is greater than 10, false otherwise
keep_these = vc.values > 10
# filtering only currencies that were greater than 10
keep_curr = vc[keep_these]
a = keep_curr.index
# filtering the dataframe to remove any of teh currencies not in our list
df = df[df['curr'].isin(a)]

#### Just keeping track of the currencies we removed in our model_dict data structure

In [8]:
remove_these = vc[vc.values <= 10].index
model_dict = {'curr_removed': list(vc[remove_these].index)}
delete_curr = list(remove_these)

#### The FX database does not have information on the following currencies
 - AED (United Arab Emirates Dirham)
 - BMD (Bermudan Dollar)
 - MXP (Mexican Peso)
 - TRY (Turkish Lira)

In [9]:
if 'TRY' not in model_dict:
    model_dict['curr_removed'].append('TRY')
    delete_curr.append('TRY')
    a = a.drop('TRY')
    
print('Model dictionary', model_dict)
print('Deleted Currencies', delete_curr)

Model dictionary {'curr_removed': ['SAR', 'BMD', 'AED', 'MXP', 'TRY']}
Deleted Currencies ['SAR', 'BMD', 'AED', 'MXP', 'TRY']


In [10]:
print("---Removing infrequent currencies from billings history---")
print('Total number of currencies in the base billings file: ', len(vc))
if len(model_dict['curr_removed'])==0:
    print('No currencies were removed, all contained 10 or more billings')
    print('Currencies in the base billings file')
    for item in a:
        print(a[item], end = " ")
else:
    print('\n Currencies were removed: ', len(model_dict['curr_removed']))

    for item in remove_these:
        print(item, ', ', end="")
        
    print("\n\n{} Remaining currencies: ".format(len(a)))
    for item in a:
        print(item, ', ', end="")

---Removing infrequent currencies from billings history---
Total number of currencies in the base billings file:  32

 Currencies were removed:  5
SAR , BMD , AED , MXP , 

27 Remaining currencies: 
USD , EUR , GBP , AUD , JPY , CHF , SEK , DKK , NOK , CAD , HKD , BRL , RUB , KRW , CLP , ARS , COP , SGD , PEN , INR , PHP , TWD , THB , MYR , IDR , NZD , ILS , 

#### Removing any of the values that are zero

In [11]:
print('This is the length of the dataframe before removing zeros: ', len(df))
df = df[df['DC_amount']!=0]
print('This is the length of the dataframe after removing zeros: ', len(df))

This is the length of the dataframe before removing zeros:  49764
This is the length of the dataframe after removing zeros:  46285


In [13]:
df.head(10)

,curr,BU,Frequency,period,config,rev_req_type,rebill_rule,Sales Type,Subscription Term,DC_amount,US_amount
1,ARS,Creative,NaN,2019-03,1Y,D,NaN,DEFERRED,1,-11291.52,-289.26
2,ARS,Creative,NaN,2019-03,1Y,D,Y3,DEFERRED,1,373766.00,9601.19
3,ARS,Creative,NaN,2019-03,1Y,D,YA,DEFERRED,12,241380.00,6194.45
4,ARS,Creative,NaN,2019-03,MTHLY,D,NaN,DEFERRED,1,-1221.00,-31.07
5,ARS,Creative,NaN,2019-03,MTHLY,D,Y3,DEFERRED,1,45799.00,1173.37
6,ARS,Creative,NaN,2019-04,1Y,D,NaN,DEFERRED,1,-40014.70,-985.24
7,ARS,Creative,NaN,2019-04,1Y,D,Y3,DEFERRED,1,1705471.00,42032.83
8,ARS,Creative,NaN,2019-04,1Y,D,YA,DEFERRED,12,1112854.12,27486.58
9,ARS,Creative,NaN,2019-04,MTHLY,D,NaN,DEFERRED,1,-14292.80,-351.04
10,ARS,Creative,NaN,2019-04,MTHLY,D,Y3,DEFERRED,1,188938.00,4653.43


#### Clearing out the Non-Revenue billings from the file
 

In [14]:
df["Sales Type"].value_counts()

DEFERRED       37628
RECOGNIZED      7183
PRO-SVC-INV     1323
NON-REV          150
Name: Sales Type, dtype: int64

In [15]:
print('Length of the dataframe before removing non-revenue billings: ', len(df))
df = df[df['Sales Type']!='NON-REV']
print('Length of the dataframe after removing non-revenue billings:  ', len(df))


Length of the dataframe before removing non-revenue billings:  46285
Length of the dataframe after removing non-revenue billings:   46135


___
## Starting to group the revenue by period, industry, etc

Grouping the data by the <b> Sales Type </b> field
 - <i>'RECOGNIZED'</i> sales are perpetual and go straight to revenue without hitting deferred 
 - <i>'PRO-SVC-INV'</i> professional services that are invoiced and go to revenue directly when invoiced
 - <i>'DEFERRED'</i> sales that will sit on the balance sheet in deferred revenue and amortize over their life
 
 #### Below we are creating a seperate dataframe for each of the Sales Types


In [37]:
rec = df[df['Sales Type']=='RECOGNIZED'].copy()
svc = df[df['Sales Type']=='PRO-SVC-INV'].copy()
dfr = df[df['Sales Type']=='DEFERRED'].copy()

print('Total number of billings:              ', len(df))
print("Number of recognized revenue billings: ", len(rec))
print("Number of service invoiced billings:   ", len(svc))
print("Number of deferred revenue billings:   ", len(dfr))

Total number of billings:               46135
Number of recognized revenue billings:  7183
Number of service invoiced billings:    1323
Number of deferred revenue billings:    37628


### Recognized Revenue

The rec.head(5) command will show the first 5 elements of the rec dataframe. To run this, simply remove the # before this line
 
The rec.tail(5) command will show the last 5 elements of the rec dataframe. To run this, simply remove the # before this line

The rec.sample(5) command will show the a random 5 elements of the rec dataframe. To run this, simply remove the # before this line

Note: Only one of these commands can be entered in a single code cell at a time. 

Note: The 5 can also be changed, but I think it caps out at 100

In [29]:
#rec.head(5)
#rec.tail(5)
#rec.sample(5)

##### Below we are grouping the rec dataframe by currency, Business Unit and Period and cleaning up the data we do not need. Since the recognized revenue go directly to revenue, there is no contract that will renew and need to be modeled in the future.

In [24]:
# testing groupby object
gb_rec = rec.groupby(['curr', 'BU', 'period'], as_index=False).sum()
gb_rec.drop(labels='Subscription Term', axis=1,inplace =True)

In [30]:
#gb_rec.head(10)
#gb_rec.tail(10)
#gb_rec.sample(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3347 entries, 0 to 3346
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   curr       3347 non-null   object 
 1   BU         3347 non-null   object 
 2   period     3347 non-null   object 
 3   DC_amount  3347 non-null   float64
 4   US_amount  3347 non-null   float64
dtypes: float64(2), object(3)
memory usage: 156.9+ KB


### Service Billings

##### Below we are grouping the svc dataframe by currency, Business Unit and Period and cleaning up the data we do not need. Since the service billings go directly to revenue, there is no contract that will renew and need to be modeled in the future.

In [32]:
gb_svc = svc.groupby(['curr', 'BU', 'period'], as_index=False).sum()
gb_svc.drop(labels='Subscription Term', axis=1,inplace =True)

In [34]:
#gb_svc.head(5)
#gb_svc.tail(5)
#gb_svc.sample(5)

### DEFERRED BILLINGS

#### Type B Billings 
Type B billings are service agreements that will have invoices submitted before the billings are reclassified to revenue. If no invoices are assigned to the billings, the billings become revenue in 12 months. Since these billings do not have a contract that will renew in the future, there is no need to model a rebillings of these service based billings


In [ ]:
dfr_b = dfr[dfr['rev_req_type']=='B'].copy()
gb_b = dfr_b.groupby(['curr', 'BU', 'period'], as_index=False).sum()
gb_b.drop(labels='Subscription Term', axis=1, inplace=True)

In [ ]:
gb_b.drop(labels='Subscription Term', axis=1, inplace=True)

## Now Type A Billings
These billings are on a billing plan. The product config tells us how long before they renew

 - '3Y' = 36 months
 - '2Y' = 24 months
 - '1Y' = 12 months
 - 'MTHLY' = 1 month
 
NOTE: There are also other fields in the 'Product Configtype ID' field that do not map well to a rebill period.
To fix this, we need to load up a different file and determine the length of the sales contract (type A no config)
 

In [ ]:
# filtering just the type A billings
dfr_a = dfr[dfr['rev_req_type']=='A'].copy()

In [ ]:
gb_a = dfr_a.groupby(['curr', 'BU', 'period',
                     'config'], as_index=False).sum()
gb_a.drop(labels='Subscription Term', axis=1, inplace = True)

In [ ]:
gb_a.head(20)

In [ ]:
gb_a['config'].value_counts()

### Below is just a check to see how large the billing types are across all periods

In [ ]:
gb_a_config = gb_a.groupby(['config'], as_index=False).sum()
gb_a_config

### These 'OCONS', 'ONORE' and 'OUNIV' data types are not actual product config IDs so we have to get them from a different data file. We are excluding these types below.

In [ ]:
config_list = ['1Y', '2Y', '3Y', 'MTHLY']
test1 = gb_a[gb_a['config'].isin(config_list)]


In [ ]:
test1.info()

## For now, lets just split this into gb_a_1Y, gb_a_2Y, gb_a_3y, gb_a_1M


In [ ]:
gb_a_1Y = test1[test1['config']=='1Y'].copy()
gb_a_2Y = test1[test1['config']=='2Y'].copy()
gb_a_3Y = test1[test1['config']=='3Y'].copy()
gb_a_1M = test1[test1['config']=='MTHLY'].copy()

In [ ]:
print('this is the lenght of type A 1M billings: ', len(gb_a_1M))
print('this is the lenght of type A 1Y billings: ', len(gb_a_1Y))
print('this is the lenght of type A 2Y billings: ', len(gb_a_2Y))
print('this is the lenght of type A 3Y billings: ', len(gb_a_3Y))

In [ ]:
gb_a_2Y.info()

# TYPE D billings
These billings have a field 'Rule For Bill Date' that determines when new billings will occur
 - Monthly:        *{Y1, Y2, Y3, Y5}*
 - Quarterly:      *YQ*
 - Every 4 months: *YT*  --NOTE: There are only 10 of these, so I am treating these as quarterly--
 - Semi-annual:    *YH*
 - Annual:         *{YA, YC}*
 - Every 2 years:  *Y4*
 - Every 3 years:  *Y7*
 
 We also need to track the type D billings that do not have a 'Rule for Bill Date'

In [ ]:
# for now just do a groupby on the type
# filtering just the type A billings
dfr_d = dfr[dfr['rev_req_type']=='D'].copy()

In [ ]:
gb_d = dfr_d.groupby(['curr', 'BU', 'period',
                     'rebill_rule'], as_index=False).sum()
gb_d.drop(labels='Subscription Term', axis=1, inplace = True)

In [ ]:
gb_d['rebill_rule'].value_counts()

In [ ]:

gb_d_mthly = gb_d[gb_d['rebill_rule'].isin(['Y1', 'Y2', 'Y3', 'YM'])].copy()
gb_d_mthly.drop(labels='rebill_rule', axis=1,inplace=True)
gb_d_mthly = gb_d_mthly.groupby(['curr', 'BU', 'period']).sum()
gb_d_mthly.reset_index(inplace=True)

gb_d_qtrly = gb_d[gb_d['rebill_rule'].isin(['YQ', 'YY', 'YT'])].copy()
gb_d_qtrly.drop(labels='rebill_rule', axis=1,inplace=True)
gb_d_qtrly = gb_d_qtrly.groupby(['curr', 'BU', 'period']).sum()
gb_d_qtrly.reset_index(inplace=True)

gb_d_semi_ann = gb_d[gb_d['rebill_rule']=='YH']

gb_d_annual = gb_d[gb_d['rebill_rule'].isin(['YA', 'YC', 'YX'])].copy()
gb_d_annual.drop(labels='rebill_rule', axis=1,inplace=True)
gb_d_annual = gb_d_annual.groupby(['curr', 'BU', 'period']).sum()
gb_d_annual.reset_index(inplace=True)


gb_d_two_yrs = gb_d[gb_d['rebill_rule']=='Y4']
gb_d_three_yrs = gb_d[gb_d['rebill_rule']=='Y7']

In [ ]:
gb_d_annual.tail(10)

In [ ]:
print('Length of monthly', len(gb_d_mthly))
print('Length of quarterly', len(gb_d_qtrly))
print('Length of four months', len(gb_d_four_mths))
print('Length of semi ann', len(gb_d_semi_ann))
print('Length of annual', len(gb_d_annual))
print('Length of two years', len(gb_d_two_yrs))
print('Length of three years', len(gb_d_three_yrs))

In [ ]:
whos

## NOW WE NEED TO BUILD A DATAFRAME THAT INTEGRATES THIS DATA

- We will have the following descriptive fields
   - Invoicing Fiscal Year-Period
   - Document Currency
   - Enterprise BU

- We will have the following fields based on rebilling rule
   - Recognized
   - Service
   - Monthly
   - Quarterly
   - Annual
   - Two Years
   - Three Years

In [ ]:
# We need to do it this way when we get to a .py file!
list_df = [gb_rec, gb_svc, gb_b,
        gb_a_1M,    gb_a_1Y,    gb_a_2Y,       gb_a_3Y, 
        gb_d_mthly, gb_d_qtrly, gb_d_semi_ann, gb_d_annual, gb_d_two_yrs, gb_d_three_yrs]

list_columns = ['recognized', 'service', 'deferred_B', 
    'deferred_1M_a', 'deferred_1Y_a', 'deferred_2Y_a', 'deferred_3Y_a',
    'deferred_1M_d', 'deferred_3M_d', 'deferred_6M_d', 'deferred_1Y_d', 'deferred_2Y_d', 'deferred_3Y_d']


In [ ]:
def sum_USD_amt(list_df, list_columns):
    total_US = []
    for df in list_df:
        total_US.append(df['US_amount'].sum())
    total_df = pd.DataFrame(index = list_columns, columns = ['US_amounts'], data=total_US)
    return total_df

In [ ]:
def merge_all_dataframes(list_df, list_columns):
    for i, df in enumerate(list_df):
        print('This is i:', i)
        #print("This is the df: ", df.head())
        print('referencing the column: ', list_columns[i])

        if i==0:
            df_merged = list_df[0].copy()
            df_merged.rename(index=str, columns={'DC_amount': list_columns[i]+'_DC', 
                                                 'US_amount': list_columns[i]+'_US'}, inplace=True)
        else:
            df_merged = merge_new_dataframe(df_merged, df, list_columns[i])

    return df_merged
    

In [ ]:
def merge_new_dataframe(old_df, new_df, new_column):
    df_merged = pd.merge(old_df, new_df, how='outer', 
                     left_on=['curr', 'BU', 'period'],
                    right_on=['curr', 'BU', 'period'])
    df_merged.rename(index=str, columns={'DC_amount': new_column+'_DC', 'US_amount': new_column+'_US'}, inplace=True)
    
    #need to drop the product configtype id for merges where the new_df is of type A
    config_str = 'config'
    rule_str = 'rebill_rule'
    if config_str in df_merged.columns:
        df_merged.drop(columns=['config'], inplace=True)
    
    if rule_str in df_merged.columns:
        df_merged.drop(columns=['rebill_rule'], inplace=True)
        
    return df_merged

In [ ]:
df = merge_all_dataframes(list_df, list_columns)

In [ ]:
def clean_df_columns(df):
    
    # clean up NaNs before adding 
    df = df.fillna(value=0)
    
    # DC columns first
    # Monthly
    df['deferred_1M_DC'] = df['deferred_1M_a_DC']+df['deferred_1M_d_DC']
    df.drop(labels=['deferred_1M_a_DC', 'deferred_1M_d_DC'], axis=1, inplace=True)
    
    # Annual
    df['deferred_1Y_DC'] = df['deferred_1Y_a_DC']+df['deferred_1Y_d_DC']
    df.drop(labels=['deferred_1Y_a_DC', 'deferred_1Y_d_DC'], axis=1, inplace=True)
    
    # Two-Year
    df['deferred_2Y_DC'] = df['deferred_2Y_a_DC']+df['deferred_2Y_d_DC']
    df.drop(labels=['deferred_2Y_a_DC', 'deferred_2Y_d_DC'], axis=1, inplace=True)
    
    #Three-Year
    df['deferred_3Y_DC'] = df['deferred_3Y_a_DC']+df['deferred_3Y_d_DC']
    df.drop(labels=['deferred_3Y_a_DC', 'deferred_3Y_d_DC'], axis=1, inplace=True)
    
    # renaming 3M and 6M
    df.rename(index=str, columns = {'deferred_3M_d_DC':'deferred_3M_DC', 
                               'deferred_6M_d_DC': 'deferred_6M_DC'}, inplace=True)

    # US columns
    # Monthly
    df['deferred_1M_US'] = df['deferred_1M_a_US']+df['deferred_1M_d_US']
    df.drop(labels=['deferred_1M_a_US', 'deferred_1M_d_US'], axis=1, inplace=True)
    
    # Annual
    df['deferred_1Y_US'] = df['deferred_1Y_a_US']+df['deferred_1Y_d_US']
    df.drop(labels=['deferred_1Y_a_US', 'deferred_1Y_d_US'], axis=1, inplace=True)
    
    # Two-Year
    df['deferred_2Y_US'] = df['deferred_2Y_a_US']+df['deferred_2Y_d_US']
    df.drop(labels=['deferred_2Y_a_US', 'deferred_2Y_d_US'], axis=1, inplace=True)
    
    # Three-Year
    df['deferred_3Y_US'] = df['deferred_3Y_a_US']+df['deferred_3Y_d_US']
    df.drop(labels=['deferred_3Y_a_US', 'deferred_3Y_d_US'], axis=1, inplace=True)
    
    # renaming 3M and 6M
    df.rename(index=str, columns = {'deferred_3M_d_US':'deferred_3M_US', 
                               'deferred_6M_d_US': 'deferred_6M_US'}, inplace=True)

    
    #cleaning up the longer column names
    df.rename(index=str, columns = {'curr': 'curr',
                               'BU':'BU',
                               'period':'period'}, inplace=True)
    
    return df

In [ ]:
df = clean_df_columns(df)

In [ ]:
df.sum()

In [ ]:
total_df = sum_USD_amt(list_df, list_columns)
total_df

In [ ]:
total_df.loc['deferred_1M_d']+total_df.loc['deferred_1M_a']

## Need to create a summary report with totals coming from every area to make sure the totals I have make sense

In [ ]:
# Make this a function to be cleaned up somehow
del dfr
del dfr_a
del dfr_b
del dfr_d
del gb_a
del gb_a_1M
del gb_a_1Y
del gb_a_2Y
del gb_a_3Y
del gb_b, 
del gb_d
del gb_svc, gb_rec, gb_d_two_yrs
del gb_d_qtrly, gb_d_semi_ann


# Now working on the ZCC billings

These billings are type D billings that did not populate the rebill_rule field of the database.

They have a 'sales document type' = 'ZCC"

The billings themselves are being created from a tableau report that looks for additions to the deferred revenue waterfall based on billings of type D and have a sales document type of ZCC

# TO BE DONE:

1. Clean up the type F billings (at least check to see if they are necessary)
2. Make a function to delete all intermediate dataframes
3. Add type A no config function
4. Add type D ZCC billings

5. Work on the forecast part of this

6. Load up FX rates

In [ ]:
# loading Adobe financial calendar and calculating period weeks
df_cal = pd.read_excel('../data/old/ADOBE_FINANCIAL_CALENDAR.xlsx', 'ADBE_cal')
df_cal['Period_Weeks'] = (df_cal['Per_End']-df_cal['Per_Start'])/np.timedelta64(1, 'W')
df_cal['Period_Weeks']=df_cal['Period_Weeks'].astype(int)
df_cal['Period_Weeks'] = df_cal['Period_Weeks']+1

In [ ]:
df_cal.head()

# Type A No Config Type Billings

This file contains type A billings that have a revenue contract start date and end date. We need to map these into the terms of our dataframe.

### Steps:
1. Rename the columns
2. This file has entries for pennies. Need to clear out anything less than $10 in absolute value
3. Determine the length of time between start date and end date
4. Group this dataframe by currency, period and BU
5. Merge this final dataframe with the larger dataframe

## NOTE: This file contains two different start date and end date columns. We need to look at all of them

In [ ]:
df_A = pd.read_excel('../data/all_billings_inputs.xlsx', sheet_name='type_A_no_config')
df_A.sample(10)

In [ ]:
df_A.columns

In [ ]:
df_A.rename(index=str, columns={'Document Currency':'curr', 
                               'Enterprise BU Desc':'BU',
                               'Invoice Fiscal Year Period Desc':'period',
                               'Rev Rec Contract End Date Hdr':'end_date_1',
                               'Rev Rec Contract End Date Item':'end_date_2',
                               'Rev Rec Contract Start Date Hdr': 'start_date_1',
                               'Rev Rec Contract Start Date Item': 'start_date_2',
                               'Completed Sales ( DC )':'DC_amount',
                               'Completed Sales': 'US_amount'
                               }, inplace=True)



In [ ]:
df_A.columns

### Dealing with the duplicate dates by taking a max

In [ ]:
df_A.head()

In [ ]:
df_A['start_date_str'] = df_A[['start_date_1','start_date_2']].max(axis=1).astype(str)
df_A['end_date_str'] = df_A[['end_date_1','end_date_2']].max(axis=1).astype(str)

In [ ]:
df_A.sample(10)

In [ ]:
df_A['start_date'] = pd.to_datetime(df_A['start_date_str'])
df_A['end_date'] = pd.to_datetime(df_A['end_date_str'])

In [ ]:
df_A.drop(labels=['end_date_1', 'end_date_2', 'start_date_1', 'start_date_2',
                  'start_date_str', 'end_date_str'], axis=1, inplace=True)

In [ ]:
df_A['month_interval']=(df_A['end_date']-df_A['start_date'])

In [ ]:
df_A.sample(10)

In [ ]:
df_A['months']= (df_A['month_interval']/ np.timedelta64(1,'M')).round(0)

In [ ]:
df_A.month_interval.describe()

### Now I need to map the months into the different integers in my dataframe


In [ ]:
list_rebills = [1, 3, 6, 12, 24, 36]
temp_rebill = np.zeros_like(df_A['months'])
for i in range(len(df_A)):
    temp_rebill[i] = min(list_rebills, key=lambda x:abs(x-df_A['months'][i]))
df_A['rebill_months']=temp_rebill


In [ ]:
df_A.sample(10)

In [ ]:
plt.scatter(df_A['months'], df_A['rebill_months'])

## Grouping the dataframe by rebill_months

In [ ]:
# drop what we dont need
df_A.drop(columns = ['start_date', 'end_date', 'month_interval', 'months'], axis=1, inplace=True)

In [ ]:
df_A.head()

In [ ]:
df_A.rename(index = str, columns={'currency': 'curr', 'Enterprise BU Desc':'BU', 
             'Invoice Fiscal Year Period Desc': 'period'}, inplace=True)


In [ ]:
df_A.head(10)

In [ ]:
#medals = df.pivot_table('no of medals', ['Year', 'Country'], 'medal')
temp_DC = df_A.pivot_table('DC_amount', ['curr', 'BU', 'period'], 'rebill_months')
temp_US = df_A.pivot_table('US_amount', ['curr', 'BU', 'period'], 'rebill_months')


In [ ]:
temp_DC

In [ ]:
temp_DC = temp_DC.fillna(0)
temp_US = temp_DC.fillna(0)

In [ ]:
temp_US

In [ ]:
temp_flat_DC = pd.DataFrame(temp_DC.to_records())
temp_flat_US = pd.DataFrame(temp_US.to_records())

In [ ]:
temp_flat_DC.info()

In [ ]:
temp_flat_DC.rename(index=str, columns={'1.0':'deferred_1M_DC', 
                               '3.0':'deferred_3M_DC',
                               '6.0':'deferred_6M_DC',
                               '12.0':'deferred_1Y_DC',
                               '24.0':'deferred_2Y_DC',
                               '36.0': 'deferred_3Y_DC'}, inplace=True)

temp_flat_US.rename(index=str, columns={'1.0':'deferred_1M_US', 
                               '3.0':'deferred_3M_US',
                               '6.0':'deferred_6M_US',
                               '12.0':'deferred_1Y_US',
                               '24.0':'deferred_2Y_US',
                               '36.0': 'deferred_3Y_US'}, inplace=True)


In [ ]:
temp_flat_DC.tail(20)

In [ ]:
df.head(30)

In [ ]:
df_test_dup = df.copy()
print(len(df_test_dup))

In [ ]:
df_test_dup =df_test_dup.drop_duplicates(subset=['curr', 'BU', 'period'])

In [ ]:
print(len(df_test_dup))

## Now we have to merge these two dataframes with the other billings dataframe

In [ ]:
''' def merge_new_dataframe(old_df, new_df, new_column):
    df_merged = pd.merge(old_df, new_df, how='outer', 
                     left_on=['curr', 'BU', 'period'],
                    right_on=['curr', 'BU', 'period'])
    df_merged.rename(index=str, columns={'DC_amount': new_column+'_DC', 'US_amount': new_column+'_US'}, inplace=True)
    
    #need to drop the product configtype id for merges where the new_df is of type A
    config_str = 'config'
    rule_str = 'rebill_rule'
    if config_str in df_merged.columns:
        df_merged.drop(columns=['config'], inplace=True)
    
    if rule_str in df_merged.columns:
        df_merged.drop(columns=['rebill_rule'], inplace=True)
        
    return df_merged
'''

In [ ]:
temp_flat_DC.head(30)

In [ ]:
df_with_A = pd.merge(df, temp_flat_DC, how='outer',
                    left_on= ['curr', 'BU', 'period'],
                    right_on=['curr', 'BU', 'period'], indicator=True, validate='one_to_one')
#df_with_A = df_with_A.fillna(value=0)

In [ ]:
#df_with_A = df_with_A.fillna({col: scalar for col in df.columns})
df_with_A = df_with_A.fillna(pd.Series(0, index=df_with_A.select_dtypes(exclude='category').columns))

In [ ]:
df_with_A.head(30)

In [ ]:
df_with_all = pd.merge(df_with_A, temp_flat_US, how='outer',
                    left_on= ['curr', 'BU', 'period'],
                    right_on=['curr', 'BU', 'period'])
#df_with_all = df_with_all.fillna(0)

In [ ]:
df_with_all = df_with_all.fillna(pd.Series(0, index=df_with_all.select_dtypes(exclude='category').columns))

In [ ]:
df_with_all.head(10)

In [ ]:
df_with_all['deferred_1M_DC']= df_with_all['deferred_1M_DC_x']+df_with_all['deferred_1M_DC_y']
df_with_all['deferred_3M_DC']= df_with_all['deferred_3M_DC_x']+df_with_all['deferred_3M_DC_y']
df_with_all['deferred_6M_DC']= df_with_all['deferred_6M_DC_x']+df_with_all['deferred_6M_DC_y']
df_with_all['deferred_1Y_DC']= df_with_all['deferred_1Y_DC_x']+df_with_all['deferred_1Y_DC_y']
df_with_all['deferred_2Y_DC']= df_with_all['deferred_2Y_DC_x']+df_with_all['deferred_2Y_DC_y']
df_with_all['deferred_3Y_DC']= df_with_all['deferred_3Y_DC_x']+df_with_all['deferred_3Y_DC_y']

df_with_all['deferred_1M_US']= df_with_all['deferred_1M_US_x']+df_with_all['deferred_1M_US_y']
df_with_all['deferred_3M_US']= df_with_all['deferred_3M_US_x']+df_with_all['deferred_3M_US_y']
df_with_all['deferred_6M_US']= df_with_all['deferred_6M_US_x']+df_with_all['deferred_6M_US_y']
df_with_all['deferred_1Y_US']= df_with_all['deferred_1Y_US_x']+df_with_all['deferred_1Y_US_y']
df_with_all['deferred_2Y_US']= df_with_all['deferred_2Y_US_x']+df_with_all['deferred_2Y_US_y']
df_with_all['deferred_3Y_US']= df_with_all['deferred_3Y_US_x']+df_with_all['deferred_3Y_US_y']

In [ ]:
df_with_all.drop(labels = ['deferred_1M_DC_x','deferred_1M_DC_y',
                        'deferred_3M_DC_x','deferred_3M_DC_y',
                        'deferred_6M_DC_x','deferred_6M_DC_y',
                        'deferred_1Y_DC_x','deferred_1Y_DC_y',
                        'deferred_2Y_DC_x','deferred_2Y_DC_y',
                        'deferred_3Y_DC_x','deferred_3Y_DC_y',
                        'deferred_1M_US_x','deferred_1M_US_y',   
                        'deferred_3M_US_x','deferred_3M_US_y',
                        'deferred_6M_US_x','deferred_6M_US_y',
                        'deferred_1Y_US_x','deferred_1Y_US_y',
                        'deferred_2Y_US_x','deferred_2Y_US_y',
                        'deferred_3Y_US_x','deferred_3Y_US_y'],
                 axis=1, inplace=True)

In [ ]:
df_with_all.head()

In [ ]:
df_with_all.columns

In [ ]:
df_with_all['deferred_1Y_US'].describe()

In [ ]:
df['deferred_1Y_US'].describe()

In [ ]:
print('sum of temp flat DC 1M', temp_flat_DC['deferred_1M_DC'].sum())
print('sum of base_df before DC 1M', df['deferred_1M_DC'].sum())
print('sum of final DC 1M', df_with_all['deferred_1M_DC'].sum())

a = temp_flat_DC['deferred_1M_DC'].sum()
b = df['deferred_1M_DC'].sum()
c = df_with_all['deferred_1M_DC'].sum()
print(c)
print(a+b)

In [ ]:
df['deferred_1M_DC'].sum()

In [ ]:
temp_flat_DC['deferred_1M_DC'].sum()

In [ ]:
df_with_all['deferred_1M_DC'].sum()

# Need to create a table that contains the total billings by DC for each dataframe and each step for auditing

 - start with all of the DC
 - then create function that appends and adds rows
 - then do the same for the DC stuff type_A
 - then check the totals


In [ ]:
df = df_with_all.copy()

In [ ]:
drop_index= df[df['period']=='2020-04'].index
df.drop(drop_index, inplace=True)

In [ ]:
# dump the pickle
#pickle.dump(df, open('../data/processed/all_billings.p', 'wb'))

# open the pickle file
df = pickle.load(open('../data/processed/all_billings.p', 'rb'))

In [ ]:
df['period'].unique()

In [ ]:
len(df)

## Loading All of the other information we need here
 - filename = currency_map

In [ ]:
df_curr_map= pd.read_excel('../data/currency_map.xlsx', sheet_name='curr_map')
df_curr_map['Country'] = df_curr_map['Country'].str.replace('\(MA\)', '', case=False)

In [ ]:
df_FX_rates = pd.read_excel('../data/FX_data.xlsx', sheet_name='to_matlab')
df_FX_rates['VOL_3M'] = df_FX_rates['VOL_3M']/100
df_FX_rates['VOL_6M'] = df_FX_rates['VOL_6M']/100
df_FX_rates['VOL_9M'] = df_FX_rates['VOL_9M']/100
df_FX_rates['VOL_1Y'] = df_FX_rates['VOL_1Y']/100

In [ ]:
df_FX_fwds = pd.read_excel('../data/FX_forward_rates.xlsx', sheet_name='forward_data', 
                          skiprows = 1, usecols="C,G")

In [ ]:
df_FX_fwds.rename(index=str, columns={'Unnamed: 2': 'curr', 'FWD REF':'forward'}, inplace=True)

## Working on the func_load_bookings now

In [ ]:
df_bookings = pd.read_excel('../data/2020_bookings_fcst_Q1.xlsx', sheet_name='bookings')


## Cleaning up the bookings data
 - remove odd strings such as '(EB)' from BU
 - (IS) from internal segment
 - etc
 
 
 NOTE: The '(' is a special character so we need to precede these with the escape character '\'

In [ ]:
df_bookings['EBU'] = df_bookings['EBU'].str.replace(' \(EB\)', '', case=False)
df_bookings['Internal Segment'] = df_bookings['Internal Segment'].str.replace('\(IS\)', '')

df_bookings['PMBU'] = df_bookings['PMBU'].str.replace('\(PMBU\)', '')
df_bookings['GEO'] = df_bookings['GEO'].str.replace('\(G\)', '')
df_bookings['Market Area'] = df_bookings['Market Area'].str.replace('\(MA\)', '')


In [ ]:
df_bookings.sample(5)

In [ ]:
df_bookings.EBU.value_counts()

In [ ]:
df_bookings.PMBU.value_counts()

In [ ]:
df_bookings.GEO.value_counts()

In [ ]:
df_bookings['Mark Segment'].value_counts()

In [ ]:
df_bookings['Type'].value_counts()

In [ ]:
df_bookings['Bookings Type'].value_counts()

In [ ]:
df_bookings['Scenario'].value_counts()

In [ ]:
df_bookings['FX'].value_counts()

In [ ]:
df_bookings.sample(5)

In [ ]:
df_bookings.drop(columns = ['Hedge', 'Mark Segment', 'Type', 'Scenario', 'FX'], inplace = True)

In [ ]:
df_bookings.rename(index=str, columns = {'EBU': 'BU', 
                                        'Internal Segment': 'segment',
                                        'PMBU': 'product',
                                        'GEO':'geo',
                                        'Market Area': 'country',
                                        'Bookings Type': 'booking_type',
                                        'value': 'US_amount'}, inplace =True)

In [ ]:
df_curr_map.head(5)

In [ ]:
df_bookings['country'].unique()

In [ ]:
list_book_ctry = df_bookings['country'].unique()


In [ ]:
list_curr_map = df_curr_map['Country'].unique()

In [ ]:
a = list(set(list_book_ctry) & set(list_curr_map))
a

In [ ]:
not_in_map = set(list_book_ctry).difference(set(list_curr_map))
not_in_map

In [ ]:
not_in_map = set()
not_in_map

In [ ]:
len(not_in_map)

In [ ]:
df_bookings = pd.merge(df_bookings, df_curr_map, how='left', left_on='country', right_on='Country')

In [ ]:
df_bookings.sample(10)

## Need to move the period_weeks from the Adobe calendar to the billings and bookings dataframes

In [ ]:
df_bookings.head(10)

# need to add the period_weeks data to both the df_bookings and df_billings dataframes

In [ ]:
df.head(10)

In [ ]:
df_cal.head(10)

In [ ]:
# need to create a column in the df_cal with both the year then '-' and the last two digits of the per_ticker

In [ ]:
df_cal['p2digit']=df_cal['Period'].astype(str)
df_cal['p2digit']=df_cal['p2digit'].str.zfill(2)
df_cal['period_match']=df_cal['Year'].astype(str) + '-' + df_cal['p2digit'].astype(str)

In [ ]:
df_cal.drop(['p2digit'],axis=1, inplace=True)
df_cal.head(10)

In [ ]:
df_cal_2_merge = df_cal.copy()
df_cal_2_merge.drop(['Year', 'Quarter', 'Period', 'Qtr_Ticker', 'Qtr_Start', 'Qtr_End', 'Per_Start',
                     'Per_Ticker','Per_End'], axis=1, inplace=True)

In [ ]:
df = df.merge(df_cal_2_merge, how='left', left_on='period', right_on='period_match')
df.drop(['period_match', '_merge'], axis=1, inplace=True)
df.head(20)

In [ ]:
df.columns

In [ ]:
df.sample(20)

In [ ]:
# now I need to merge the period weeks with the df_fcst

In [ ]:
df_billings=df_billings.sort_values(['curr', 'BU', 'period'], ascending = (True, True, True))

input_df_dict = {'billings':df, 
              'ADBE_cal':df_cal,
              'bookings': df_bookings,
              'FX_forwards': df_FX_fwds,
              'FX_rates': df_FX_rates
                }

pickle.dump(input_df_dict, open('../data/processed/all_inputs.p', 'wb'))

def join_new_dataframe(old_df, new_df, new_column):
    df_joined = old_df.join(new_df, how='outer')
    df_joined.rename(index=str, columns={'DC_amount': new_column+'_DC', 'US_amount': new_column+'_US'}, inplace=True)
    df_joined.fillna(value=0, inplace=True)
    
    #need to drop the product configtype id for merges where the new_df is of type A
    config_str = 'config'
    rule_str = 'rebill_rule'
    if config_str in df_joined.columns:
        df_joined.drop(columns=['config'], inplace=True)
    
    if rule_str in df_joined.columns:
        df_joined.drop(columns=['rebill_rule'], inplace=True)
        
    return df_joined


def join_all_dataframes(list_df, list_columns):
    for i, df in enumerate(list_df):
        print('This is i:', i)
        print('referencing the column: ', list_columns[i])
        #print(df.columns)
        # setting the index to be 'curr', 'BU' and 'period'
        df.set_index(['curr', 'BU', 'period'], drop=True, inplace=True, verify_integrity=True)
        print(df.head(4))
        #if i==0:
            #df_joined = df.copy()
            #df_joined = list_df[0].copy()
            #df_joined.rename(index=str, columns={'DC_amount': list_columns[i]+'_DC', 
            #                                     'US_amount': list_columns[i]+'_US'}, inplace=True)
        #else:
            #df_joined = join_new_dataframe(df_joined, df, list_columns[i])

    #return df_joined
    x=1
    return x 